In this test we try parsing Fedlex documents of type DRAFT with Docling.

In [1]:
DATA_VERSION = "2025-11-17"
DATA_FILE_NAME = f"consultation-documents-preprocessed-{DATA_VERSION}.parquet"

LANGUAGES = {
    "de",
    # "fr",
    # "it",
}

FROM_YEAR = 2010

In [2]:
%load_ext autoreload
%autoreload 2

import datetime
import pathlib
import sys

import docling.document_converter
import duckdb

REPOSITORY_ROOT = (pathlib.Path().cwd() / ".." / "..").resolve()
sys.path.append(str(REPOSITORY_ROOT))

/Users/vita/Code/demokratis/demokratis-ml/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import demokratis_ml.data.loading
from research.lib import data_access

In [4]:
PREPROCESSED_DATA_FILE = REPOSITORY_ROOT / "data" / "dataframes" / DATA_FILE_NAME
data_access.ensure_dataframe_is_available(PREPROCESSED_DATA_FILE)

2025-11-19 10:36:12,022 - INFO - File /Users/vita/Code/demokratis/demokratis-ml/data/dataframes/consultation-documents-preprocessed-2025-11-17.parquet already exists locally.


In [5]:
db_conn = duckdb.connect(":memory:")

rel_docs = demokratis_ml.data.loading.filter_documents(
    db_conn.from_parquet(str(PREPROCESSED_DATA_FILE)),
    only_languages=LANGUAGES,
    only_consultations_since=datetime.date(FROM_YEAR, 1, 1),
    only_document_types={"DRAFT"},
)
df = rel_docs.to_df()
df = df.loc[df["political_body"] == "ch"]
df

,document_uuid,consultation_identifier,consultation_start_date,consultation_end_date,consultation_title,consultation_description,document_source_url,consultation_url,consultation_topics,consultation_internal_tags,...,political_body,latest_stored_file_uuid,document_source,consultation_topics_label_source,document_type_label_source,document_content_plain,stored_file_path,stored_file_mime_type,stored_file_hash,__index_level_0__
0,1CAJoURQMxKeqCdQ4jLniS,7bt3hz2w,2021-10-01,2022-01-17,Verordnung des BAZG über die Bekämpfung von G...,Mit der am 19. März 2021 durch das Parlament b...,https://fedlex.data.admin.ch/filestore/fedlex....,https://www.demokratis.ch/vernehmlassung/7bt3hz2w,"[administration, finance, security]","[{'created_at': 2024-11-04 15:11:24, 'name': '...",...,ch,1CM3wffQLMxGCHhjJ6PKmR,fedlex,manual,fedlex,«$$e-seal» \n«$$QrCode» \n \n2021-… \n«%ASFF_Y...,2024/1/1/1CM3wffPbDu7fUxY1BS22e.pdf,application/pdf,b8816738ad84d806e55723a313613097bed2752f,0
1,1CAJoURregQSBjKq3Pp2mL,a49pncfq,2021-12-03,2022-03-18,Änderung der Verordnung über Fernmeldedienste ...,In Vollzug von Artikel 48a FMG über die Sicher...,https://fedlex.data.admin.ch/filestore/fedlex....,https://www.demokratis.ch/vernehmlassung/a49pncfq,"[communications, it, security]","[{'created_at': 2025-01-27 11:30:14, 'name': '...",...,ch,1CM3wg6vz2uQ3oZqEQcf7s,fedlex,manual,fedlex,[Signature] \n[QR Code] \n \n1 \nVerordnung \n...,2024/2/22/1CM3wg6vjrx6u67JD8PayD.pdf,application/pdf,8fb68017e53055498c277ed8afe60d56ee2935aa,21
2,1CAJoUSHBj4qdYJPe3PFcd,tcspwfam,2021-10-11,2022-01-25,"Teilrevisionen der Raumplanungsverordnung, der...","Das Eidgenössische Departement für Umwelt, Ver...",https://fedlex.data.admin.ch/filestore/fedlex....,https://www.demokratis.ch/vernehmlassung/tcspwfam,"[energy, spatial_planning]","[{'created_at': 2025-02-11 13:16:43, 'name': '...",...,ch,1CM3wgUgRrjEFontegfgyd,fedlex,manual,fedlex,2021-… \n \nVerordnung \n \n \n \nVernehmlass...,2024/3/43/1CM3wgUcDxsq2yKqdZsaGe.pdf,application/pdf,98a60f2d0b0b0ce18b5ce82a1e9df248c3629bb9,42
3,1CAJoUSHBj4qdYJPedbRja,tcspwfam,2021-10-11,2022-01-25,"Teilrevisionen der Raumplanungsverordnung, der...","Das Eidgenössische Departement für Umwelt, Ver...",https://fedlex.data.admin.ch/filestore/fedlex....,https://www.demokratis.ch/vernehmlassung/tcspwfam,"[energy, spatial_planning]","[{'created_at': 2025-02-11 13:16:43, 'name': '...",...,ch,1CM3wgwv3CkfxghjShq2NP,fedlex,manual,fedlex,1 \nRaumplanungsverordnung \n \nVernehmlassung...,2024/3/52/1CM3wgwuYYGtzYePP6ETCu.pdf,application/pdf,0ec7371d74bfec45d6b022706fe669bee7005ded,51
4,1CAJoUSHBj4qdYJPepDs22,tcspwfam,2021-10-11,2022-01-25,"Teilrevisionen der Raumplanungsverordnung, der...","Das Eidgenössische Departement für Umwelt, Ver...",https://fedlex.data.admin.ch/filestore/fedlex....,https://www.demokratis.ch/vernehmlassung/tcspwfam,"[energy, spatial_planning]","[{'created_at': 2025-02-11 13:16:43, 'name': '...",...,ch,1CM3wgyzoKLLbozGmFEg75,fedlex,manual,fedlex,2021-… \n \nVerordnung \n \n \n \nVernehmlass...,2024/3/55/1CM3wgyzJL2fSpmyQyW3Uj.pdf,application/pdf,fc01f855d3a18f17ee7b3b9add690ded4d9d2764,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,1CUsdXTeZ4Uj78TZQAfebr,bc4kn9t9,2025-11-06,2026-02-20,Bundesgesetz über den CO2-Grenzausgleich bei d...,"Die Kommission für Umwelt, Raumplanung und Ene...",https://fedlex.data.admin.ch/filestore/fedlex....,https://www.demokratis.ch/vernehmlassung/bc4kn9t9,"[economics, energy, environment]","[{'created_at': 2025-11-05 10:56:10, 'name': '...",...,ch,1CUsdbGeNAdj22vDHAEtV1,fedlex,manual,fedlex,2025-... \n \nBundesgesetz \nVorentwurf \nübe...,2025/5284/57613/1CUsdbGe83dVtmtRcGLxst.pdf,application/pdf,17bb61787ba2ae2812fe6cc59dd398360d3fceb4,54806
3079,1CV6tCxFGgAyAVycGzR7pe,6axbhyxa,2025-11-12,2026-02-28,Änderung der Ausweisverordnung und der Verordn...,Ende 2026 soll zusätzlich zur heutigen Identit...,https://fedlex.data.admin.ch/filestore/fedlex....,https://www.demokratis.ch

In [6]:
document_url = df["document_source_url"].sample().iloc[0]
document_url

'https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/dl/proj/6017/6/cons_1/doc_8/de/pdf-a/fedlex-data-admin-ch-eli-dl-proj-6017-6-cons_1-doc_8-de-pdf-a.pdf'

In [7]:
converter = docling.document_converter.DocumentConverter()
result = converter.convert(document_url)

2025-11-19 10:36:19,256 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-19 10:36:19,312 - INFO - Going to convert document batch...
2025-11-19 10:36:19,313 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-19 10:36:19,321 - INFO - Loading plugin 'docling_defaults'
2025-11-19 10:36:19,323 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-19 10:36:19,332 - INFO - Loading plugin 'docling_defaults'
2025-11-19 10:36:19,336 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-11-19 10:36:20,066 - INFO - Auto OCR model selected ocrmac.
2025-11-19 10:36:20,072 - INFO - Accelerator device: 'mps'
2025-11-19 10:36:22,169 - INFO - Accelerator device: 'mps'
2025-11-19 10:36:22,850 - INFO - Processing document fedlex-data-admin-ch-eli-dl-proj-6017-6-cons_1-doc_8-de-pdf-a.pdf
2025-11-19 10:36:36,296 - INFO - Finished converting document fedlex-data-admin-ch

In [8]:
print(result.document.export_to_markdown())

<!-- image -->

## Energieverordnung (EnV)

vom …

Der Schweizerische Bundesrat, gestützt auf das Energiegesetz vom 30. September 2016 1 (EnG),

verordnet:

## 1. Kapitel: Gegenstand

## Art. 1

Diese Verordnung regelt:

- a. den Herkunftsnachweis und die Stromkennzeichnung;
- b. die Raumplanung im Zusammenhang mit dem Ausbau erneuerbarer Energien;
- c. die Einspeisung netzgebundener Energie und den Eigenverbrauch;
- d. die wettbewerblichen Ausschreibungen für Effizienzmassnahmen;
- e. die Geothermie-Garantien und -Erkundungsbeiträge;
- f. die Entschädigung für Sanierungsmassnahmen bei Wasserkraftanlagen;
- g. den Netzzuschlag;
- h. die sparsame und effiziente Energienutzung in Gebäuden und Unternehmen;
- i. die Fördermassnahmen im Energiebereich;
- j. die internationale Zusammenarbeit im Anwendungsbereich des EnG;
- k. die Untersuchung der Wirkungen und die Datenbearbeitung.

## 2. Kapitel: Herkunftsnachweis und Stromkennzeichnung

## 1. Abschnitt: Herkunftsnachweis

## Art. 2 Pflicht

In [9]:
print(result.document.export_to_html())

<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8"/>
<title>fedlex-data-admin-ch-eli-dl-proj-6017-6-cons_1-doc_8-de-pdf-a</title>
<meta name="generator" content="Docling HTML Serializer"/>
<style>
    html {
        background-color: #f5f5f5;
        font-family: Arial, sans-serif;
        line-height: 1.6;
    }
    body {
        max-width: 800px;
        margin: 0 auto;
        padding: 2rem;
        background-color: white;
        box-shadow: 0 0 10px rgba(0,0,0,0.1);
    }
    h1, h2, h3, h4, h5, h6 {
        color: #333;
        margin-top: 1.5em;
        margin-bottom: 0.5em;
    }
    h1 {
        font-size: 2em;
        border-bottom: 1px solid #eee;
        padding-bottom: 0.3em;
    }
    table {
        border-collapse: collapse;
        margin: 1em 0;
        width: 100%;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
    }
    th {
        background-color: #f2f2f2;
        font-weight: bold;
    }
    figure {
       